In [8]:
import os
import yaml
import json
import csv

# === Percorsi base ===
metrics_dir = "metrics"
catalog_path = "catalogs/demo_catalog.json"
output_csv = "metrics_catalog_map.csv"

# === 1️⃣ Crea dizionario metric_id -> (category, description) ===
metrics_data = {}

for root, dirs, files in os.walk(metrics_dir):
    for file in files:
        if file.endswith((".yaml", ".yml")):
            path = os.path.join(root, file)
            with open(path, "r") as f:
                try:
                    content = yaml.safe_load(f)
                    metric_id = content.get("id")
                    category = content.get("category")
                    description = content.get("description", "")
                    if metric_id:
                        metrics_data[metric_id] = {
                            "category": category,
                            "description": description,
                            "control_id": None
                        }
                except Exception as e:
                    print(f"Errore leggendo {path}: {e}")

# === 2️⃣ Carica il catalogo e crea mappa metric_id -> control_id ===
with open(catalog_path, "r") as f:
    catalogs = json.load(f)

def extract_controls(catalog_section):
    controls_map = {}
    for category in catalog_section.get("categories", []):
        for control in category.get("controls", []):
            if "controls" in control:
                for sub_control in control["controls"]:
                    for metric in sub_control.get("metrics", []):
                        metric_id = metric.get("id")
                        control_id = sub_control.get("id")
                        if metric_id:
                            controls_map[metric_id] = control_id
    return controls_map

controls_map = {}
for catalog in catalogs:
    controls_map.update(extract_controls(catalog))

# === 3️⃣ Associa metriche ai controlli ===
for metric_id, info in metrics_data.items():
    if metric_id in controls_map:
        info["control_id"] = controls_map[metric_id]

# === 4️⃣ Scrivi il CSV finale ===
with open(output_csv, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["metric_id", "control_id", "category", "description"])
    for metric_id, info in metrics_data.items():
        writer.writerow([
            metric_id,
            info.get("control_id", ""),
            info.get("category", ""),
            info.get("description", "")
        ])

print(f"✅ File CSV generato: {output_csv}")


✅ File CSV generato: metrics_catalog_map.csv
